<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import math
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Input, GRU, Dense, LSTM, Flatten, Reshape, Lambda, CuDNNLSTM, Dropout
import matplotlib.pyplot as plt

# Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'
icd9_code = '99592'
minMaxScaler = dict()
itemMeans = dict()

# Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

# Load Patient data




In [0]:
table = dict()

sepsis = pd.read_csv('/home/adarsh/Documents/sepsis/sepsis3-df.csv')
sepsis.sort_values(by='icustay_id',na_position='first')
table['sepsis'] = sepsis

def get_pandas(table):
  query = "SELECT * FROM "+table+";"
  table = pd.read_sql(query,conn)
  table.sort_values(by='icustay_id',na_position='first')
  return table

tabs = ['uo','vitals','labs','ventsettings','vd','bgart','gcs']

for tab in tabs:
  table[tab] = get_pandas(tab)

In [0]:
sepsis_dataset = table['sepsis']
for tab in tabs:
  b = table[tab]
  sepsis_dataset = pd.merge(sepsis_dataset,b,left_index=True,right_index=True)